## Import TensorFlow and other libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

In [ ]:
from GAF.preprocessing.features_extractors.raw_extractor import RawExtractor
from GAF.preprocessing.data_translator.gaf_translator import GafTranslator
from GAF.preprocessing.preprocessor import Preprocessor

TIME_PER_SAMPLE = 2
WINDOWS = 4
BATCH_SIZE = 800
RESAMPLE_MS = TIME_PER_SAMPLE * 1000 // WINDOWS
SAMPLES_PER_MINUTE = 60
PATH = f"/sise/yos-group/royhersh/data/processed/fixed_{SAMPLES_PER_MINUTE}"
ANOMALIES_PATH = f"/sise/yos-group/royhersh/anomalies/processed/fixed_{SAMPLES_PER_MINUTE}"
AXIS_WINDOWS_AMOUNT = (
    TIME_PER_SAMPLE * SAMPLES_PER_MINUTE
)  # 4 windows * 0.500 s/window * 60 points-in-sample/s = 120 points-in-sample
AXES = ["x", "y", "z", "tot"]

extractor = RawExtractor(resample_amount=RESAMPLE_MS)
preprocess = Preprocessor(
    extractor,
    translators=[GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS)],
    packed_windows=WINDOWS,
    path=PATH,
)
anomalies_preprocess = Preprocessor(
    extractor,
    translators=[GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS)],
    packed_windows=WINDOWS,
    path=ANOMALIES_PATH,
)

In [ ]:
import datetime
import pathlib
import os

OUTPUT_FILE = (
    "gaf_autoencoders/{extractor}/{time}/".format(
        extractor=str(extractor), time=datetime.datetime.now().strftime("%d.%m.%y")
    )
    + "/{file_name}.{file_type}"
)
os.makedirs(pathlib.Path(OUTPUT_FILE).parent, exist_ok=True)
FILE_TYPE = "h5"

In [ ]:
sensor = "gyroscope"


def load_dataset(preprocess, sensor):
    data, labels, info = preprocess.load_dataset_sensor(sensor)

    print("[*] Spliting test/train")
    return preprocess.create_dataset(data, labels, info, flatten=False)

In [ ]:
anomaly_train_data, _, anomaly_test_data, _, _, _ = load_dataset(anomalies_preprocess, sensor)
train_data, train_labels, test_data, test_labels, _, _ = load_dataset(preprocess, sensor)

Plot a normal sample. 

In [ ]:
def simple_plot(title, data):
    data = np.array(data).reshape(AXIS_WINDOWS_AMOUNT, AXIS_WINDOWS_AMOUNT)
    plt.clf()
    plt.title(title)
    fig = plt.figure(1, dpi=100)

    width_ratios = (0.4, 7, 0.4)
    height_ratios = (0.4, 7)
    # width = 10
    # height = width * sum(height_ratios) / sum(width_ratios)
    # plt.gcf().set_size_inches(width, height)
    plt.title(title)
    gs = fig.add_gridspec(
        2,
        3,
        width_ratios=width_ratios,
        height_ratios=height_ratios,
        left=0.1,
        right=0.9,
        bottom=0.1,
        top=0.9,
        wspace=0.1,
        hspace=0.1,
    )

    # Plot the Gramian angular fields on the bottom right
    ax_gasf = fig.add_subplot(gs[1, 1])
    ax_gasf.imshow(data, cmap="rainbow", origin="lower", extent=[0, 4 * np.pi, 0, 4 * np.pi])
    ax_gasf.set_xticks([])
    ax_gasf.set_yticks([])

    # Add colorbar
    im = ax_gasf.imshow(data, cmap="rainbow", origin="lower", extent=[0, 4 * np.pi, 0, 4 * np.pi])
    ax_cbar = fig.add_subplot(gs[1, 2])
    fig.colorbar(im, cax=ax_cbar)
    plt.show()

In [ ]:
for i in range(3):
    for axis in range(len(AXES)):
        simple_plot(f"A gyroscope {axis} sample {i}", train_data[i][axis])

Plot an anomalous ECG.

In [ ]:
for i in range(3):
    for axis in range(len(AXES)):
        simple_plot(f"An anomalous gyroscope {axis} sample {i}", anomaly_train[i][axis])

### Build the model

In [ ]:
class CustomAccuracy(tf.keras.losses.Loss):
    def __init__(self, threshold=0.0001):
        super().__init__()
        self._threshold = threshold
        self._inv_threshold = 1 / threshold

    def call(self, y_true, y_pred):
        diff = tf.reduce_mean(tf.math.abs(y_pred - y_true))
        if diff > self._threshold:
            return tf.square(self._inv_threshold * diff)
        return diff


class AnomalyDetector(Model):
    def __init__(self):
        super(AnomalyDetector, self).__init__()
        self.encoder = tf.keras.Sequential(
            [
                layers.Dense(300, activation="relu"),
                layers.Dropout(0.1),
                layers.Dense(200, activation="relu"),
                layers.Dropout(0.1),
                layers.Dense(120, activation="relu"),
            ]
        )

        self.decoder = tf.keras.Sequential(
            [
                layers.Dense(200, activation="relu"),
                layers.Dropout(0.1),
                layers.Dense(300, activation="relu"),
                layers.Dropout(0.1),
                layers.Dense(AXIS_WINDOWS_AMOUNT, activation="sigmoid"),
            ]
        )

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def save_model(self, file_name: str, extra_detail: str = ""):
        self.encoder.save(
            OUTPUT_FILE.format(file_name=f"{file_name}_encoder{extra_detail}", file_type=FILE_TYPE),
            save_format=FILE_TYPE,
        )
        self.decoder.save(
            OUTPUT_FILE.format(file_name=f"{file_name}_decoder{extra_detail}", file_type=FILE_TYPE),
            save_format=FILE_TYPE,
        )

In [ ]:
class AnomalyDetectors:
    def __init__(self, train_data, test_data, axes=AXES):
        self._train_data = train_data
        self._test_data = test_data
        self._axes = axes
        self._autoencoders = [AnomalyDetector() for _ in range(len(axes))]
        for autoencoder in self._autoencoders:
            autoencoder.compile(optimizer="adam", loss=tf.keras.losses.MeanSquaredError())
            # autoencoder.compile(optimizer='adam', loss=CustomAccuracy(), metrics=['mae', 'mse'])

    def encoder(self, x):
        output = []
        for index in range(len(self._autoencoders)):
            tmp = [array[index] for array in x]
            output.append(self._autoencoders[index].encoder(tmp).numpy())
        return output

    def decoder(self, x):
        output = []
        for index in range(len(self._autoencoders)):
            tmp = [array[index] for array in x]
            output.append(self._autoencoders[index].decoder(tmp).numpy())
        return output

    def call(self, x):
        output = []
        for index in range(len(self._autoencoders)):
            tmp = [array[index] for array in x]
            output.append(self._autoencoders[index].call(tmp))
        return output

    def train(self, epochs):
        histories = [[] for _ in range(len(self._autoencoders))]
        length_train = len(self._train_data)
        length_test = len(test_data)
        print(f"Training data {length_train}")
        print(f"Test data {length_test}")
        # test_data._batch_size = BATCH_SIZE * 5
        validation_data = [[] for _ in range(len(self._autoencoders))]
        for axis in range(len(self._autoencoders)):
            test_cur = np.asarray([data[axis] for self._ in self._test_data], dtype=np.float32)
            validation_data[axis] = (test_cur, test_cur)
            train_cur = np.asarray([data[axis] for data in self._train_data], dtype=np.float32)
            print(f"train_load {i}/{length_train} {axis}")
            history = self._autoencoders[axis].fit(
                train_cur,
                train_cur,
                epochs=epochs,
                shuffle=False,
                validation_data=validation_data[axis],
            )
            histories[axis].append(history)
        return histories

    def save_model(self, file_name):
        for index in range(len(self._autoencoders)):
            self._auto_encoders[index].save_model(sensor, index)

In [ ]:
def load_model(model_date):
    global OUTPUT_FILE
    OUTPUT_FILE = (
        "autoencoders/{preprocessor}/{time}/".format(preprocessor=str(preprocess), time=model_date)
        + "/{file_name}.{file_type}"
    )
    model_file_name = f"{sensor}_{extractor._ms_resample}x{preprocess._packed_windows}"
    autoencoder.load_model(model_file_name)


new_model = False
# load_model("18.03.23")

In [ ]:
new_model = True
autoencoder = AnomalyDetectors(train_data, test_data)

Notice that the autoencoder is trained using only the normal ECGs, but is evaluated using the full test set.

In [ ]:
histories = autoencoder.train(epochs=200)

In [ ]:
for history_list in histories:
    total_loss = []
    total_val_loss = []
    for history in history_list:
        total_loss.extend(history.history["loss"])
        total_val_loss.extend(history.history["val_loss"])
    plt.plot(total_loss, label="Training Loss")
    plt.plot(total_val_loss, label="Validation Loss")
    plt.legend()

You will soon classify an ECG as anomalous if the reconstruction error is greater than one standard deviation from the normal training examples. First, let's plot a normal ECG from the training set, the reconstruction after it's encoded and decoded by the autoencoder, and the reconstruction error.

In [ ]:
train_batch = train_dataset.get_train()[0]
test_batch = train_dataset.get_test()[0]
encoded_data = autoencoder.encoder(test_batch)
decoded_data = autoencoder.decoder(encoded_data)

In [ ]:
def plot_decoded_original(title, test_data, decoded_data):
    simple_plot(f"Input {title}", test_data)
    simple_plot(f"Reconstructed {title}", decoded_data)

In [ ]:
for i in range(3):
    for axis in range(len(AXES)):
        plot_decoded_original(f"{sensor} {axis} sample {i}", test_batch[i][axis], decoded_data[i][axis])

Create a similar plot, this time for an anomalous test example.

In [ ]:
anomaly_train_batch = anomaly_dataset.get_train()[0]
anomaly_test_batch = anomaly_dataset.get_test()[0]
anomaly_encoded_data = autoencoder.encoder(anomaly_train_batch)
anomaly_decoded_data = autoencoder.decoder(anomaly_encoded_data)

In [ ]:
for i in range(3):
    for axis in range(len(AXES)):
        plot_decoded_original(
            f"{sensor} {axis} anomalous sample {i}",
            anomaly_test_batch[i][axis],
            anomaly_decoded_data[i][axis],
        )

### Detect anomalies

Detect anomalies by calculating whether the reconstruction loss is greater than a fixed threshold. In this tutorial, you will calculate the mean average error for normal examples from the training set, then classify future examples as anomalous if the reconstruction error is higher than one standard deviation from the training set.


Plot the reconstruction error on normal ECGs from the training set

In [ ]:
def calculate_loss(model, data):
    print(data.shape)
    reconstructions = model.call(data)
    print(np.array(reconstructions).shape)
    # custom_acc = CustomAccuracy()
    custom_acc = tf.keras.losses.MeanSquaredError()
    amount, axes, _data1, _data2 = data.shape
    loss = []
    for i in range(amount):
        tmp_loss = 0
        for axis in range(axes):
            tmp_loss += custom_acc.call(reconstructions[i][axis], data[i][axis])
        loss.append(tmp_loss)
    return loss

In [ ]:
train_loss = calculate_loss(autoencoder, train_batch)

plt.hist(train_loss, bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
print(train_loss[0])
for axis in range(len(AXES)):
    print(
        f"custom loss function result example: {i}",
        custom_acc.call(reconstructions[axis][0], anomaly_train_batch[0][axis]),
    )

Choose a threshold value that is one standard deviations above the mean.

In [ ]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Benign threshold: ", threshold)

In [ ]:
test_loss = calculate_loss(autoencoder, test_batch)

print(
    "Span: ",
    np.mean(test_loss) - np.std(test_loss),
    np.mean(test_loss) + np.std(test_loss),
)

plt.hist(test_loss, bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

Note: There are other strategies you could use to select a threshold value above which test examples should be classified as anomalous, the correct approach will depend on your dataset. You can learn more with the links at the end of this tutorial. 

If you examine the reconstruction error for the anomalous examples in the test set, you'll notice most have greater reconstruction error than the threshold. By varing the threshold, you can adjust the [precision](https://developers.google.com/machine-learning/glossary#precision) and [recall](https://developers.google.com/machine-learning/glossary#recall) of your classifier. 

In [ ]:
anomaly_test_loss = calculate_loss(autoencoder, anomaly_train_batch)

anomaly_lower_span = np.mean(anomaly_test_loss) - np.std(anomaly_test_loss)
anomaly_upper_span = np.mean(anomaly_test_loss) + np.std(anomaly_test_loss)
print("Span: ", anomaly_lower_span, anomaly_upper_span)

anomaly_threshold = anomaly_lower_span

plt.hist(anomaly_test_loss, bins=50)
plt.xlabel("Anomaly test loss")
plt.ylabel("No of examples")
plt.show()

Classify an ECG as an anomaly if the reconstruction error is greater than the threshold.

In [ ]:
from collections.abc import Iterable
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def memoize(function):
    memo = {}

    def wrapper(*args):
        hash_str = ""
        for arg in args:
            arg_type = type(arg)
            if arg_type == bytes or arg_type == str:
                hash_str += f"{str(arg[0])}{str(arg[-1])}{len(arg)}_"
            elif arg_type == list or arg_type == tuple:
                # print("[x] Cannot hash well list and tuples")
                tmp_arg = arg
                try:
                    while True:
                        hash_str += f"{type(tmp_arg[0])}{len(tmp_arg)}_"
                        tmp_arg = tmp_arg[0]
                except Exception:
                    pass
            elif arg_type == np.ndarray or arg_type == np.array:
                # print("[x] Cannot hash well arrays")
                hash_str += f"{str(arg.max())}{str(arg.min())}{arg.shape}_"
            elif arg_type == object:
                hash_str += f"{arg_type}_"
            else:
                hash_str += f"{str(arg)}_"
        # print(hash_str)
        if hash_str in memo:
            return memo[hash_str]
        else:
            rv = function(*args)
            memo[hash_str] = rv
            return rv

    return wrapper

In [ ]:
@memoize
def predict(model, data, threshold):
    loss = calculate_loss(model, data)
    return tf.math.less(loss, threshold)


# cannot memoize without threshold with the following memoize function
def get_stats(predictions, labels):
    return f"Accuracy = {accuracy_score(labels, predictions)}"


def get_stats_all(predictions, labels):
    return f"""{get_stats(predictions, labels)}
Precision = {precision_score(labels, predictions)}
Recall = {recall_score(labels, predictions)}
F1 = {f1_score(labels, predictions)}
"""

In [ ]:
from itertools import repeat, chain


def evaluate(model, threshold):
    output = f"\n[*] Calculating result with threshold {threshold}"
    predications_benign = predict(model, test_batch, threshold)
    labels_benign = list(repeat(True, len(predications_benign)))
    stats_benign = get_stats(predications_benign, labels_benign)
    output += "\nBenign test performance: \n" + stats_benign

    predications_anomaly = predict(model, anomaly_train_batch, threshold)
    labels_anomaly = list(repeat(False, len(predications_anomaly)))
    stats_anomaly = get_stats(predications_anomaly, labels_anomaly)
    output += "\nAnomaly train performance: \n" + stats_anomaly

    predications_anomaly_test = predict(model, anomaly_test_batch, threshold)
    labels_anomaly_test = list(repeat(False, len(predications_anomaly_test)))
    stats_anomaly = get_stats(predications_anomaly_test, labels_anomaly_test)
    output += "\nAnomaly test performance: \n" + stats_anomaly

    predications_total = list(chain(predications_benign, predications_anomaly, predications_anomaly_test))
    labels_total = list(chain(labels_benign, labels_anomaly, labels_anomaly_test))
    stats_total = get_stats_all(predications_total, labels_total)
    output += "\nTotal performance: \n" + stats_total
    return output

In [ ]:
# print(evaluate(autoencoder, threshold))
print(evaluate(autoencoder, anomaly_threshold))

In [ ]:
if new_model:
    model_file_name = f"{sensor}_{extractor._ms_resample}x{preprocess._packed_windows}"
    with open(OUTPUT_FILE.format(file_name=model_file_name, file_type="log"), "w+") as f:
        f.write(f"Train: <data: {len(train_dataset.get_train())}>, Test:<data: {len(train_dataset.get_test())}>\n")
        f.write(
            f"Anomalies: <data: {len(anomaly_dataset.get_train())}>, Test:<data: {len(anomaly_dataset.get_test())}>\n"
        )
        f.write(evaluate(autoencoder, threshold))
        f.write(evaluate(autoencoder, anomaly_threshold))
    autoencoder.save_model(model_file_name)
    print("[+] Model saved")
new_model = False

In [ ]:
import os

job_cancel_str = "scancel " + os.environ["SLURM_JOBID"]
os.system(job_cancel_str)